In [1]:
from transformers import BertTokenizer,BertForMaskedLM
import torch 
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
text = "The capital of Iran is [MASK]."
text = "Yesterday, I played [MASK] in the stadium"
text = "Yesterday, I played [MASK] in the [MASK]."
text = "I'm studying [MASK] learning in my computer class."
text = "I'm a very [MASK] player in football."
text = "He drived a [MASK]."
text = "I love playing [MASK]."
text = "I am [MASK] english."
inputs = tokenizer(text, return_tensors="pt")
print(tokenizer.mask_token_id)

103


In [7]:
with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
print(inputs.input_ids == tokenizer.mask_token_id)
print(mask_token_index)


tensor([[False, False, False,  True, False, False, False]])
tensor([3])


In [17]:
import numpy as np
import editdistance
import pandas as pd

N = 20
word = 'diano'
token_ids = np.argpartition(logits[0, mask_token_index], -N, axis=-1)[0,-N:]
scores = logits[0, mask_token_index][0, token_ids]
df = pd.DataFrame({'token_id':token_ids, 'scores':scores})
df['word'] = df['token_id'].apply(lambda row: tokenizer.decode([row]))
df['edit_distance'] = df['word'].apply(lambda row: editdistance.eval(word,row))
df.sort_values(by = 'scores', ascending=False)
# display(df)
# print(df['edit_distance'].idxmin())


,token_id,scores,word,edit_distance
5,2025,11.230057,not,4
13,4092,11.103004,speaking,6
14,1999,10.102517,in,3
11,4083,7.643354,learning,6
17,2053,7.444749,no,3
6,2019,7.016780,an,3
9,19376,6.977960,fluent,5
15,2200,6.632783,very,5
12,4252,6.566546,teaching,6
8,2069,6.517703,only,5
